In [1]:
# library import-------------------------------------------


import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""


import time
from pathlib import Path

import FinanceDataReader as fdr
import pandas as pd
import pandas_ta as ta
import torch
from pykrx import stock
from sklearn.metrics import classification_report
from tqdm import tqdm

from tsai.all import *

my_setup()
torch.cuda.is_available()

os             : Linux-5.4.0-122-generic-x86_64-with-glibc2.17
python         : 3.8.13
tsai           : 0.2.23
fastai         : 2.7.8
fastcore       : 1.5.14
torch          : 1.10.0
n_cpus         : 16
device         : cuda (NVIDIA GeForce RTX 3060)


True

In [2]:
# data load###########################################################    

#npy load
predictDayAbove=1
testDateStop='2022-07-01'
npyPath=Path(f"preprocess2_date{testDateStop}_day{predictDayAbove}")
sampleListKey=["X","y","trainIndex","testIndex","ticker","yDate", "unlabelX","unlabelTicker","unlabelDate"]
sampleListDict={key:[] for key in sampleListKey}
for key in sampleListKey:
    sampleListDict[key] = np.load(npyPath/f'pykrx_1_{key}.npy')
      
        
#splits
sampleListDict["testIndex"]=list(sampleListDict["testIndex"])
sampleListDict["trainIndex"]=list(sampleListDict["trainIndex"])
splits=[sampleListDict["trainIndex"],sampleListDict["testIndex"]]

# X y
baseNum = 0.002
X = sampleListDict["X"]
y = sampleListDict["y"]
y_cat = sampleListDict["y"] > baseNum


# class_weights
n_1 = np.count_nonzero(y_cat[sampleListDict["trainIndex"]])
n_0 = len(sampleListDict["trainIndex"]) - n_1
w_0 = (n_0 + n_1) / (2.0 * n_0)
w_1 = (n_0 + n_1) / (2.0 * n_1)
class_weights = torch.FloatTensor([w_0, w_1]).cuda()


#test
XTest=sampleListDict["X"][sampleListDict["testIndex"]]
yTest=sampleListDict["y"][sampleListDict["testIndex"]]
tickerTest=sampleListDict["ticker"][sampleListDict["testIndex"]]
dateTest=sampleListDict["yDate"][sampleListDict["testIndex"]]
lastClosePriceTest=XTest[:,0,-1]

#unlabel
XUnlabel=sampleListDict["unlabelX"]
tickerUnlabel=sampleListDict["unlabelTicker"]
dateUnlabel=sampleListDict["unlabelDate"]
lastClosePriceUnlabel=XUnlabel[:,0,-1]



#model test func
def printResult(test_probas,yPred,yTarget,yReg):
    expectedEarningPerDayWithPred=np.mean(yReg[yPred])*100
    expectedEarningPerDayWithAllBuy=np.mean(yReg)*100
    print(f"probas_min:{np.min(test_probas):.4f} probas_max:{np.max(test_probas):.4f}")
    print(f"truePredCount:{yReg[yPred].shape[0]}")
    print(f"expectedEarningPerDayWithPred:{expectedEarningPerDayWithPred:.4f}%")
    print(f"expectedEarningPerDayWithAllBuy:{expectedEarningPerDayWithAllBuy:.4f}%")
    print(classification_report(yTarget, yPred))

    
    

In [3]:
#load model
PATH = Path('./models/regression2.pkl')
learn = load_learner(PATH, cpu=True)


In [4]:
#predicted result's mean earning
 
test_probas, _, preds = learn.get_X_preds(X[sampleListDict["testIndex"]])

    
base=0.002
test_probas=test_probas.numpy()[:,0]
yPred=test_probas>base
yTarget=y[sampleListDict["testIndex"]]>base
yReg=y[sampleListDict["testIndex"]]

printResult(test_probas,yPred,yTarget,yReg)


probas_min:-0.0193 probas_max:0.0140
truePredCount:10571
expectedEarningPerDayWithPred:0.1520%
expectedEarningPerDayWithAllBuy:-0.1052%
              precision    recall  f1-score   support

       False       0.59      0.89      0.71     50513
        True       0.46      0.14      0.21     35376

    accuracy                           0.58     85889
   macro avg       0.52      0.51      0.46     85889
weighted avg       0.54      0.58      0.50     85889

